In [18]:
import threading as th
import random
import time
import numpy as np

## Programa 1

In [2]:
class Tortuga(th.Thread):
    

    def __init__(self):
        th.Thread.__init__(self)
        self.probabilidad = 0
        self.recorrido_tortuga = 1
        self.control = True
    
    def stop(self):
        self.control = False
    
    def run(self):
        while (self.control):
            self.probabilidad = random.randint(1,100)
            if self.probabilidad <= 50:
                self.recorrido_tortuga += 3
                if self.recorrido_tortuga > 70:
                    self.recorrido_tortuga = 70
                print("Posicion {}: {}T".format(self.recorrido_tortuga, " "*self.recorrido_tortuga))

            elif self.probabilidad <= 70:
                self.recorrido_tortuga -= 6
                if self.recorrido_tortuga < 1:
                    self.recorrido_tortuga = 1
                print("Posicion {}: {}T".format(self.recorrido_tortuga, " "*self.recorrido_tortuga))
            else:
                self.recorrido_tortuga += 1
                if self.recorrido_tortuga > 70:
                    self.recorrido_tortuga = 70
                print("Posicion {}: {}T".format(self.recorrido_tortuga, " "*self.recorrido_tortuga))
            time.sleep(1)

In [3]:
class Liebre(th.Thread):
   
    def __init__(self):
        th.Thread.__init__(self)
        self.probabilidad = 0
        self.recorrido_liebre = 1
        self.control = True
    
    def stop(self):
        self.control = False
        
    def run(self):
        while (self.control):
            self.probabilidad = random.randint(1,100)
            if self.probabilidad <= 20:
                self.recorrido_liebre = self.recorrido_liebre
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))

            elif self.probabilidad <= 40:
                self.recorrido_liebre += 9
                if self.recorrido_liebre > 70:
                    self.recorrido_liebre = 70
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))

            elif self.probabilidad <= 50:
                self.recorrido_liebre -= 12
                if self.recorrido_liebre < 1:
                    self.recorrido_liebre = 1
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))

            elif self.probabilidad <= 80:
                self.recorrido_liebre += 1
                if self.recorrido_liebre > 70:
                    self.recorrido_liebre = 70
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))
            else:
                self.recorrido_liebre -= 2
                if self.recorrido_liebre < 1:
                    self.recorrido_liebre = 1
                print("Posicion {}: {}L".format(self.recorrido_liebre, " "*self.recorrido_liebre))

            time.sleep(1)

In [4]:
recorrido_liebre = 1
recorrido_tortuga = 1

lock = th.Lock()

tortuga = Tortuga()
liebre = Liebre()

tortuga.start()
liebre.start()

while True:
    if liebre.recorrido_liebre == 70 and tortuga.recorrido_tortuga == 70:
        print("Empate")
        tortuga.stop()
        liebre.stop()
        tortuga.join()
        liebre.join()
        break
        
    if liebre.recorrido_liebre == 70:
        print("Liebre ganó")
        tortuga.stop()
        liebre.stop()
        tortuga.join()
        liebre.join()
        break
        
    if tortuga.recorrido_tortuga == 70:
        print("Tortuga ganó")
        tortuga.stop()
        liebre.stop()
        tortuga.join()
        liebre.join()
        break

Posicion 4:     T
Posicion 1:  L
Posicion 1:  LPosicion 7:        T

Posicion 1:  L
Posicion 1:  T
Posicion 1:  TPosicion 2:   L

Posicion 11:            LPosicion 2:   T

Posicion 1:  TPosicion 12:             L

Posicion 13:              LPosicion 4:     T

Posicion 14:               L
Posicion 7:        T
Posicion 14:               L
Posicion 10:           T
Posicion 23:                        L
Posicion 13:              T
Posicion 21:                      LPosicion 14:               T

Posicion 15:                T
Posicion 30:                               L
Posicion 9:          T
Posicion 31:                                L
Posicion 12:             T
Posicion 32:                                 L
Posicion 6:       T
Posicion 30:                               L
Posicion 9:          T
Posicion 30:                               L
Posicion 12:             TPosicion 18:                   L

Posicion 18:                   LPosicion 15:                T

Posicion 16:                 T


## Programa 2

In [109]:
def multiplicacion_matrices(A, B):
    
    C = []
    
    Am = len(A) 
    An = len(A[0])
    
    Bm = len(B)
    Bn = len(B[0])
    
    if An != Bm:
        return False
    
    for k in range(Am):
        fila_C = []
        
        
        for i in range(Bn):
            sumador = 0
            for j in range(Bm):
                sumador += A[k,j] * B[j,i]
            
            fila_C.append(sumador)
        C.append(fila_C)
    
    return C

In [137]:
A = np.random.randint(10, size=(50, 50))
B = np.random.randint(10, size=(50, 50))
 
t = time.time()
C = multiplicacion_matrices(A,B)
print("Tiempo : {}".format(time.time() - t))

Tiempo : 0.14170169830322266


In [138]:
def obtencion_submatrices(A):
    if len(A) % 4 == 0:
        k = len(A) // 4
        submatrices = [A[i:i+k] for i in range(0,len(A),k)]
    else:
        k = len(A) // 4
        residuo = len(A) - k*4
        sin_residuo = len(A) - residuo
        submatrices = [A[i:i+k] for i in range(0, sin_residuo, k)]
        submatrices.append(A[sin_residuo:])
    return submatrices

In [139]:
submatrices_A = obtencion_submatrices(A)
submatrices_C = []

In [140]:
def multiplicacion_matrices_paralela(B, i):
    
    global submatrices_A,submatrices_C 
    
    A = submatrices_A[i]
    C = []
    
    Am = len(A) 
    An = len(A[0])
    
    Bm = len(B)
    Bn = len(B[0])
    
    if An != Bm:
        return False
    
    for k in range(Am):
        fila_C = []
        
        
        for i in range(Bn):
            sumador = 0
            for j in range(Bm):
                sumador += A[k,j] * B[j,i]
            
            fila_C.append(sumador)
        C.append(fila_C)
    
    submatrices_C.append(C)

In [141]:
candado  = th.Lock()

t = time.time()

hilos = [th.Thread(target = multiplicacion_matrices_paralela, args = (B, i)) for i in range(len(submatrices_A))]

for h in hilos:
    h.start()
    
for h in hilos:
    h.join()
    
print("Tiempo : {}".format(time.time() - t))

Tiempo : 0.1198110580444336


In [142]:
for c in submatrices_C:
    for cc in c:
        print()
        print(cc)


[969, 960, 894, 977, 837, 1049, 940, 1020, 1018, 889, 1064, 956, 816, 813, 1127, 927, 892, 932, 777, 1077, 977, 766, 1026, 1048, 894, 998, 913, 1056, 961, 941, 885, 1066, 885, 833, 954, 993, 818, 938, 1072, 957, 933, 937, 988, 1034, 935, 1059, 1087, 1005, 854, 1136]

[1061, 1166, 941, 1152, 980, 1221, 1138, 1308, 1094, 913, 1204, 1085, 938, 1060, 1168, 1005, 1046, 1085, 1034, 1228, 1104, 867, 1226, 1075, 1028, 1065, 1079, 1073, 1082, 899, 973, 1141, 1065, 949, 1104, 1108, 956, 998, 1334, 988, 1168, 1116, 1033, 1166, 996, 1094, 1112, 1118, 1144, 1383]

[1019, 1101, 860, 1100, 1037, 1082, 1033, 1128, 1019, 818, 1212, 1142, 990, 929, 1212, 1149, 970, 1133, 941, 1215, 1014, 870, 1095, 1133, 996, 957, 1040, 1198, 1009, 972, 1001, 1113, 1027, 804, 1084, 1038, 993, 1091, 1350, 1078, 1099, 1014, 1064, 972, 947, 1120, 1214, 1021, 933, 1297]

[1046, 1082, 954, 932, 979, 1000, 944, 1008, 1012, 771, 951, 1061, 923, 1059, 1036, 1051, 974, 1094, 894, 1077, 1039, 790, 998, 1014, 857, 923, 885, 1086,

In [143]:
for c in C:
    print()
    print(c)


[969, 960, 894, 977, 837, 1049, 940, 1020, 1018, 889, 1064, 956, 816, 813, 1127, 927, 892, 932, 777, 1077, 977, 766, 1026, 1048, 894, 998, 913, 1056, 961, 941, 885, 1066, 885, 833, 954, 993, 818, 938, 1072, 957, 933, 937, 988, 1034, 935, 1059, 1087, 1005, 854, 1136]

[1061, 1166, 941, 1152, 980, 1221, 1138, 1308, 1094, 913, 1204, 1085, 938, 1060, 1168, 1005, 1046, 1085, 1034, 1228, 1104, 867, 1226, 1075, 1028, 1065, 1079, 1073, 1082, 899, 973, 1141, 1065, 949, 1104, 1108, 956, 998, 1334, 988, 1168, 1116, 1033, 1166, 996, 1094, 1112, 1118, 1144, 1383]

[1019, 1101, 860, 1100, 1037, 1082, 1033, 1128, 1019, 818, 1212, 1142, 990, 929, 1212, 1149, 970, 1133, 941, 1215, 1014, 870, 1095, 1133, 996, 957, 1040, 1198, 1009, 972, 1001, 1113, 1027, 804, 1084, 1038, 993, 1091, 1350, 1078, 1099, 1014, 1064, 972, 947, 1120, 1214, 1021, 933, 1297]

[1046, 1082, 954, 932, 979, 1000, 944, 1008, 1012, 771, 951, 1061, 923, 1059, 1036, 1051, 974, 1094, 894, 1077, 1039, 790, 998, 1014, 857, 923, 885, 1086,